In [1]:
import tvm
from tvm import relay
from tvm.contrib import graph_runtime
from tvm.relay import create_executor
import numpy as np
from tvm.relay.prelude import Prelude

In [8]:
mytype = relay.TensorType((), "int32")
a = relay.Var("a", mytype)
cond = a < relay.const(10, dtype="int32")
sb = relay.ScopeBuilder()
with sb.if_scope(cond):
    sb.ret(relay.const(42, dtype="int32"))
with sb.else_scope():
    sb.ret(relay.const(6, dtype="int32"))
net = relay.Function([a], sb.get())
mod = relay.Module.from_expr(net)
mod = relay.transform.InferType()(mod)
params={}

opt_level = 1
target = "llvm"
with relay.build_config(opt_level=opt_level):
    graph, lib, params = relay.build_module.build(
        mod, target, params=params)

TVMError: Traceback (most recent call last):
  [bt] (8) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::backend::GraphRuntimeCodegen::Codegen(tvm::relay::Function)+0x10b) [0x7f8677f9b41b]
  [bt] (7) /home/ubuntu/tvm/build/libtvm.so(std::_Function_handler<void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), tvm::runtime::TypedPackedFunc<tvm::Map<tvm::relay::Expr, tvm::Array<tvm::Array<tvm::Integer, void>, void>, void, void> (tvm::relay::Function const&)>::AssignTypedLambda<tvm::Map<tvm::relay::Expr, tvm::Array<tvm::Array<tvm::Integer, void>, void>, void, void> (*)(tvm::relay::Function const&)>(tvm::Map<tvm::relay::Expr, tvm::Array<tvm::Array<tvm::Integer, void>, void>, void, void> (*)(tvm::relay::Function const&))::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}>::_M_invoke(std::_Any_data const&, tvm::runtime::TVMArgs&&, tvm::runtime::TVMRetValue*&&)+0x4e) [0x7f8677f8648e]
  [bt] (6) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::GraphPlanMemory(tvm::relay::Function

In [2]:
mod = relay.Module()
p = Prelude(mod)
nil = p.nil
cons = p.cons
hd = p.hd

mytype = relay.TensorType((), "float32")
a = relay.Var("a", mytype)
b = cons(a, cons(a, nil()))
c = hd(b)
func = relay.Function([a], c)
mod["main"] = func


opt_level = 1
target = "llvm"
with relay.build_config(opt_level=opt_level):
    graph, lib, params = relay.build_module.build(
        mod, target, params={})


# intrp = create_executor(mod=mod)
# aa = tvm.nd.array(np.random.rand())
# print(aa)
# res = intrp.evaluate()(aa)
# print(res)

TVMError: Traceback (most recent call last):
  [bt] (8) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::ExprVisitor::VisitExpr_(tvm::relay::CallNode const*)+0x1a9) [0x7fcc568c7479]
  [bt] (7) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::ExprVisitor::VisitExpr(tvm::relay::Expr const&)+0x83) [0x7fcc568cbce3]
  [bt] (6) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::ExprFunctor<void (tvm::relay::Expr const&)>::VisitExpr(tvm::relay::Expr const&)+0x445) [0x7fcc5672fca5]
  [bt] (5) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::ExprVisitor::VisitExpr_(tvm::relay::CallNode const*)+0x3a) [0x7fcc568c730a]
  [bt] (4) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::ExprVisitor::VisitExpr(tvm::relay::Expr const&)+0x83) [0x7fcc568cbce3]
  [bt] (3) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::ExprFunctor<void (tvm::relay::Expr const&)>::VisitExpr(tvm::relay::Expr const&)+0x445) [0x7fcc5672fca5]
  [bt] (2) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::TypeVarEVisitor::VisitExpr_(tvm::relay::ConstructorNode cons